In [1]:

import numpy as np
import scipy as sp
import h5py

import os

import SimData as sim_data
import Simulator as sim_system
import Optimization as opt
import SurrogateDE as mDE

In [2]:


###* Input files and function

reactions_file = "reactionsSimpleV1.json"

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

initial_state_dict = {'O_F': 0.1, 'O2_F':0.1 ,'O_S': 0.1, 'Vdb_S':0.1, 
                      'Odb_S': 0.1, 'CO_F': 0.1, 'CO2_F':0.1, 'CO_S': 0.1, 
                      'COdb_S': 0.0}

###* Functions for the data transformation
def compute_flux(const_dict, exp_dict, specie, molar_mass):
    den = exp_dict.get(specie, 0.0)
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux


def compute_remaining_flux(const_dict, exp_dict, molar_mass): 
    den = exp_dict['N'] - exp_dict['O'] - exp_dict['CO']
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux

####? EavgMB data extracted from the Booth et al. 2019 paper
p_data_exp = [0.2, 0.3, 0.4, 0.5, 0.6, 0.75, 1.5]
EavgMB_data = [1.04, 0.91, 0.87, 0.83, 0.77, 0.5, 0.001]
interpolator = sp.interpolate.interp1d(p_data_exp, EavgMB_data, kind='linear', fill_value=0.001, bounds_error=False)


transformations_exp = {
    'Tw':       lambda const_dict, exp_dict: exp_dict['Tw'] + 273.15,
    'fluxO' :   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O', 0.016),
    'fluxO2' :  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O2', 0.032),
    'fluxO3' :  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O3', 0.048),
    'fluxC':    lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'C', 0.012),
    'fluxCO':   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO', 0.028),
    'fluxCO2':  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO2', 0.048),
    'EavgMB':   lambda const_dict, exp_dict: interpolator(exp_dict['pressure']).item(),
    'Ion':      lambda const_dict, exp_dict: 1e14 * exp_dict["current"]
}

output_folder_path = "Buffer_Data"
exp_data_file = "Experimental_data_CO_Jorge.hdf5"

In [3]:


def func_optimization(params):
    
    A_O, B_O, E_O, \
    A_CO, B_CO, E_CO = params
    
    nu_d_mod = lambda T: 1e15 * (A_O + B_O * np.exp(E_O/(const_dict['R'] * T)))
    nu_d_mod_2 = lambda T: 1e15 * (A_CO + B_CO * np.exp(E_CO/(const_dict['R'] * T)))
    
    dict_mod_vec = [
    {"id": 2, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 10, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 31, "rate": None, "model_dict": {"nu_d": nu_d_mod_2}}
    ]
    
    return dict_mod_vec

loss_function = lambda exp, teo: np.mean((np.reciprocal(exp)*(exp-teo))**2)

exp_file = os.path.join(output_folder_path, exp_data_file)
sim = sim_system.Simulator(reactions_file, const_dict, exp_file, initial_state_dict, transformations_exp=transformations_exp)


  d[CO2_F]/dt = -CO2_F*r_29 + r_28*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_F*O_S*r_39 - CO_F*r_31 - CO_F*r_33 - 0.02*CO_F*r_35*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) + r_30*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_S]/dt = CO_F*r_35*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) - CO_S*O_F*r_38 - CO_S*r_36 + r_32*(-CO_S - O_S - Odb_S - Vdb_S + 1.0)
  d[O2_F]/dt = -O2_F*O_F*r_15 - O2_F*r_10 - O2_F*r_12 - O2_F*r_14 + r_9*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_S*O_F*r_38 - O2_F*O_F*r_15 - 2*O_F**2*r_8 - 0.02*O_F*O_S*r_7 - 0.02*O_F*Odb_S*r_27 - 0.02*O_F*Vdb_S*r_26 - O_F*r_11 - O_F*r_2 - O_F*r_4 - 0.02*O_F*r_5*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) + r_1*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_S]/dt = -CO_F*O_S*r_39 - O_F*O_S*r_7 + O_F*r_5*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) - O_S*r_16 - O_S*r_17 - O_S*r_37 - O_S*r_6 + r_3*(-CO_S - O_S - Odb_S - Vdb_S + 1.0)
  d[Odb_S]/dt = -O_F*Odb_S*r_27 + O_F*Vdb_S*r_26 - Odb_S*r_23 - Odb_S*r_24 - Odb

In [4]:

if __name__ == '__main__':
    optimizer = opt.Optimizer(sim, func_optimization, loss_function)

    global_bounds = [(1e-5, 1.0), (1e-8, 1.0), (5, 30), \
                    (1e-5, 1.0), (1e-8, 1.0), (5, 30)]

    config = {
        "bounds": global_bounds,
        "nb_calls": 3, # 5
        "de_maxiter": 10,
        "de_num_iterations": 2,
        "de_max_generations": 100,
        "top_k": 3,
        "local_attempts": 1,
        "epsilon_local": 1e-2
    }
    
    ##3 objectvive fucno
    
    def my_simulator(params):
        return optimizer.objective_function(params)
    
    
    opt_DE = mDE.SurrogateDE(my_simulator, global_bounds, popsize=5, max_gens=50,
                       surrogate_every=5, surrogate_min_points=50)
    
    best_x, best_f = opt_DE.run()
    print("Best solution:", best_x, best_f)
    
# differential_evolution step 50: f(x)= 0.041662857183863554


f_trial:  0.8601398675205206
f_trial:  0.860077679962801
f_trial:  0.8184435766653502
f_trial:  0.8601452457359132
f_trial:  0.8600422719998391
f_trial:  0.752522500454735
f_trial:  0.8600747160551586
f_trial:  0.8535884751551058
f_trial:  0.7568182960811728
f_trial:  0.8601477860339032
f_trial:  0.8537306158160057
f_trial:  0.842573582225151
f_trial:  0.8575591473918664
f_trial:  0.8601436150898487
f_trial:  0.8600805022103896
f_trial:  0.8137252040806259
f_trial:  0.8601000466660901
f_trial:  0.8584711278061227
f_trial:  0.8601173732903583
f_trial:  0.8601406422295855
f_trial:  0.8601323708556382
f_trial:  0.8305659070662097
f_trial:  0.2583197070469246
f_trial:  0.8601415912339889
f_trial:  4552219.525877803
f_trial:  0.6171039960339163
f_trial:  0.7706050698460022
f_trial:  0.858017298254325
f_trial:  0.8578893095652387
f_trial:  0.8599262194773915
False
Gen   1: best_f = 0.258320
f_trial:  0.8078528681762449
f_trial:  0.860116668081066
f_trial:  0.8228346140501528
f_trial:  0.8025

/opt/miniconda3/envs/LLM/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Gen   5: best_f = 0.227316
f_trial:  0.6109280277947574
f_trial:  0.7774317184799127
f_trial:  5.711413352104219
f_trial:  0.4630594852091654
f_trial:  3.577059197640927
f_trial:  0.22742309320687157
f_trial:  0.27990443131025766
f_trial:  0.2624433282352256
f_trial:  0.8601285514174289
f_trial:  3.995891043728225
f_trial:  0.25946328908174704
f_trial:  0.8600182985392459
f_trial:  0.40346163202310736
f_trial:  0.8600118723415285
f_trial:  0.9290427653682923
f_trial:  0.8601486132530964
f_trial:  4.582208082601571
f_trial:  9.56699099940911
f_trial:  0.4636191239979888
f_trial:  0.3022272645524587
f_trial:  0.3502241898841936
f_trial:  0.8601354174227579
f_trial:  0.8601171610457303
f_trial:  0.7824877819491916
f_trial:  0.26854743541931947
f_trial:  0.8601025446204565
f_trial:  0.8589255140341541
f_trial:  0.2545744849425065
f_trial:  0.8601324694982112
f_trial:  0.8275889342219901
False
Gen   6: best_f = 0.227316
f_trial:  0.6944948730956932
f_trial:  0.3502309859562295
f_trial:  0.3

/opt/miniconda3/envs/LLM/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/miniconda3/envs/LLM/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Gen  10: best_f = 0.161010
f_trial:  0.2746249375937423
f_trial:  0.7431523829734947
f_trial:  0.7207762052198476
f_trial:  0.26479937316482405
f_trial:  0.8601415650243212
f_trial:  0.8334060537109752
f_trial:  0.26609138828853973
f_trial:  1.7233948841194158
f_trial:  0.42568334538162717
f_trial:  0.8601384624256324
f_trial:  0.7991025249013913
f_trial:  0.25833755753486043
f_trial:  0.6881201552630128
f_trial:  0.860078424948096
f_trial:  0.8589646100989378
f_trial:  0.2869083244880642
f_trial:  0.3434820497408647
f_trial:  0.2687531851663706
f_trial:  6.397490901250795
f_trial:  0.22270083705437904
f_trial:  561.5112614874951
f_trial:  0.3301493007641862
f_trial:  0.8757003612325095
f_trial:  0.8598872690546248
f_trial:  0.26541187765894036
f_trial:  0.29012996928404705
f_trial:  9.052323509271146
f_trial:  0.3989112714605224
f_trial:  0.2052908851580724
f_trial:  0.8601464969677064
False
Gen  11: best_f = 0.161010
f_trial:  7.036040393726373
f_trial:  0.7656463443244569
f_trial:  

...

proposals:  [[5.17982024e-01 2.17095642e-01 5.15038641e+00 9.99309174e-01
  2.65180688e-02 1.83413184e+01]
 [8.20640456e-02 1.38267700e-01 5.40361367e+00 3.95913046e-01
  9.16672276e-01 2.14563341e+01]
 [4.56412466e-01 2.89528316e-01 5.65534738e+00 5.04157676e-01
  4.20438245e-02 8.79283317e+00]
 [9.02988155e-01 2.53135949e-01 2.94321756e+01 2.47967838e-01
  9.75345373e-01 1.57858679e+01]
 [7.30829271e-01 4.53790143e-02 5.05439262e+00 5.77873757e-01
  9.25977944e-01 2.05346511e+01]
 [6.43150948e-01 4.53644387e-01 1.78355456e+01 2.90291625e-01
  4.36012958e-01 1.65832115e+01]
 [8.87497132e-01 3.74239760e-01 1.22022600e+01 7.55725233e-01
  9.64137870e-01 8.20109111e+00]
 [2.95583853e-01 4.08232596e-01 1.40113144e+01 4.48454011e-01
  3.63712543e-01 2.85682620e+01]
 [9.22656948e-01 8.81971591e-01 2.92613960e+01 6.73079262e-01
  7.50652684e-01 1.21162405e+01]
 [2.13703484e-02 1.14271810e-01 5.11730400e+00 9.35010342e-01
  3.60182813e-01 5.75395018e+00]]
Gen  40: best_f = 0.160728
f_trial:  

/opt/miniconda3/envs/LLM/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


proposals:  [[ 0.517552    0.22757561  6.09426384  0.71783357  0.30903076 26.82918185]
 [ 0.72473081  0.72414821  6.24762818  0.9534563   0.29987717 18.27494763]
 [ 0.88172899  0.29176997 25.01145577  0.72432227  0.67900126 29.90361587]
 [ 0.82852842  0.23434941  5.01658323  0.41998747  0.22059231 18.56550041]
 [ 0.25611204  0.47274089 17.05268279  0.34019071  0.4837878   5.17186041]
 [ 0.76513743  0.10821304 12.47380412  0.75131866  0.47173556 15.8583153 ]
 [ 0.74622571  0.76616514 29.93516968  0.90567206  0.03636931  6.84365489]
 [ 0.70953914  0.2613006   6.84500026  0.24210077  0.12534901 20.28495736]
 [ 0.28606738  0.84642145 14.41202788  0.35759756  0.31974298 29.24957094]
 [ 0.96251842  0.25690291 14.2890895   0.3987244   0.64149064 12.22130507]]
Gen  45: best_f = 0.158584
f_trial:  0.8562069900693294
f_trial:  0.2553119360299928
f_trial:  0.8600938687999076
f_trial:  4728465.008520897
f_trial:  0.7715067232865886
f_trial:  0.16095908099799394
f_trial:  4728465.077575472
f_trial:

/opt/miniconda3/envs/LLM/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


proposals:  [[6.99048471e-01 5.78267974e-01 5.01663250e+00 4.75832195e-01
  5.28456679e-01 1.81935473e+01]
 [4.06168291e-01 2.32359740e-01 2.49550152e+01 1.12798286e-02
  6.53403830e-02 2.11183162e+01]
 [9.03027571e-01 1.02488901e-01 2.44105333e+01 7.99086196e-01
  3.28347438e-01 1.83280554e+01]
 [9.06338832e-01 2.46138514e-01 2.99618884e+01 3.99225551e-01
  2.84387138e-01 9.69415388e+00]
 [2.95096584e-01 3.73884451e-01 8.42417213e+00 4.22867610e-01
  9.97161205e-01 7.05171138e+00]
 [1.92071477e-01 2.58420179e-01 2.14901554e+01 1.39518992e-01
  8.50451634e-01 1.94530649e+01]
 [4.74407072e-01 4.43147019e-01 6.78138831e+00 3.61642801e-01
  3.52111252e-01 2.03754298e+01]
 [3.83547152e-01 4.34546298e-01 2.38616337e+01 5.27649149e-01
  2.73379665e-01 2.61826475e+01]
 [6.41165366e-01 5.31649621e-01 6.91377975e+00 8.83730618e-01
  6.07577257e-01 2.97192416e+01]
 [9.14673980e-01 2.26809277e-01 2.93021975e+01 4.08406596e-01
  6.50628499e-01 1.59948281e+01]]
Gen  50: best_f = 0.156369
Best solut

In [5]:
# DE resul: [7.54561369e-02 3.30911944e-04 1.80716671e+01 6.55878140e-02
#  9.49664603e-01 5.15676107e+00] | 0.04027624574740497

# ####* Test default 

# import matplotlib.pyplot as plt

# params = np.array([0.01634, 1.67e-4, 19.75])



def do_plots(params):


    loss = optimizer.objective_function(params)
    print("PARAMS: ", params, "Loss: ", loss)

    frac_solutions_arr, gammas_results_arr, gammas_sum_arr, gamma_exp_data = optimizer.solve_simulations_updated(params)
    exp_data_arr = optimizer.exp_data_arr

    inv_Tnw_vec = np.array([300/vec['Tnw'] for vec in exp_data_arr])

    plt.figure
    plt.plot(inv_Tnw_vec, gamma_exp_data, '.', label="Exp")
    plt.plot(inv_Tnw_vec, gammas_sum_arr, '.', label="Sim")
    plt.legend()
    plt.xlabel(r"$300/T_{nw}$")
    plt.ylabel(r"$\gamma$")

    current_vec = np.array([vec['current'] for vec in exp_data_arr])
    Tw_vec = np.array([vec['Tw'] for vec in exp_data_arr])

    current_unique = np.unique(current_vec)
    Tw_unique = np.unique([vec['Tw'] for vec in exp_data_arr])


    A, B = np.meshgrid(current_unique, Tw_unique)
    combinations = np.stack([A.ravel(), B.ravel()], axis=1)

    for comb in combinations:
        current = comb[0]
        Tw = comb[1]

        idx_valid = np.logical_and(current_vec == current, Tw_vec == Tw)
        data_arr = exp_data_arr[idx_valid]
        
        pressure_vec = np.array([vec['pressure'] for vec in data_arr])
        gamma_exp = gamma_exp_data[idx_valid]
        gamma_sim = gammas_sum_arr[idx_valid]
        
        plt.figure()
        plt.plot(pressure_vec, gamma_exp*1e4, 's', label="Exp")
        plt.plot(pressure_vec, gamma_sim*1e4, 's', label="Sim")
        plt.title(f"I = {current} and Tw = {Tw}")
        plt.xlabel("Pressure")
        plt.ylabel(r"$\gamma$")
        plt.legend()
        plt.grid()
        plt.ylim(1, 14)
        plt.show()



In [6]:

params_1 = np.array([0.01634, 1.67e-4, 19.75])
do_plots(params_1)


ValueError: not enough values to unpack (expected 6, got 3)

In [ ]:





params_2 = np.array([2.39375716e-02, 7.11846527e-04, 1.64416626e+01])
do_plots(params_2)

In [ ]:

params_3 = np.array([8.32874763e-03, 8.32658918e-04, 1.61121534e+01])
do_plots(params_3)

In [ ]:

# def nu_d_func(T, params):
#     A, B, E = params
    
#     nu_d_mod = 1e15 * (A + B * np.exp(E * np.reciprocal(const_dict['R'] * T)))
    
#     return nu_d_mod


# T_space = np.linspace(-20+273.15, 50+273.15, 200)

# vec_1 = np.array([nu_d_func(T, params_1) for T in T_space])
# vec_2 = np.array([nu_d_func(T, params_2) for T in T_space])
# vec_3 = np.array([nu_d_func(T, params_3) for T in T_space])

# plt.plot(T_space, vec_1, label="Paper")
# plt.plot(T_space, vec_2, label="DE 1")
# plt.plot(T_space, vec_3, label="DE 2")
# plt.legend()
# plt.xlabel("Tw")
# plt.ylabel(r"$\nu_{d}$")